In [1]:
import pickle
from collections import namedtuple

In [2]:
with open('cache/mass_shootings_2015.pickle', 'rb') as infile:
    look_up = pickle.load(infile)

In [3]:
print(look_up.keys())

dict_keys([('location',), ('participant',), ('location', 'participant')])


In [4]:
CONFUSION_TUPLE=('location', 'participant')
MIN_CONFUSION=1

In [10]:
def lookup_and_merge(look_up, confusion_key):
    nice_combos=[]
    Candidate=namedtuple('Candidate', 'granularity sf meanings num_answer_e ')
    for granularity in (look_up[confusion_key]):
        if granularity[0]=='state':
            continue
        for sf in look_up[confusion_key][granularity]:
            len_combo=len(look_up[confusion_key][granularity][sf])
            meanings=look_up[confusion_key][granularity][sf]
            if(len_combo>MIN_CONFUSION):
                c=Candidate(granularity=granularity, sf=sf, meanings=meanings, num_answer_e=len_combo)
                nice_combos.append(c)
    return nice_combos

candidates=lookup_and_merge(look_up, CONFUSION_TUPLE)
for c in candidates:
    print(c.granularity)

('city', 'full_name')
('city', 'first')
('city', 'first')
('city', 'first')
('city', 'first')
('city', 'last')
('city', 'last')
('city', 'last')


In [36]:
def compute_metrics(c, look_up, confusion_key, index):
    my_set=set()
    confusion=confusion_key[index]
    meanings=look_up[(confusion,)][c.granularity[index]][c.sf[index]]
    for m in meanings.values():
        for incident in m:
            my_set.add(incident)
#    print(c.sf[index], my_set)
    num_noisy_e=len(my_set)
    ns_ratio=num_noisy_e/c.num_answer_e
    oa=len(meanings)
    print('The %s %s has %d meanings over %d incidents. OA is %d, Noise-to-Signal ratio is %f' % (confusion, c.sf[index],oa,num_noisy_e, oa, ns_ratio))
    return ns_ratio, oa
            
    
def compute_confusion_metrics(look_up, confusion_key, candidates):
    for c in candidates:
        print('CANDIDATE: %s' % str(c.sf))
        ns_loc, oa_loc=compute_metrics(c, look_up, confusion_key, 0)
        ns_part, oa_part=compute_metrics(c, look_up, confusion_key, 1)
        score=(ns_loc + oa_loc + ns_part + oa_part)/4
        print("Question score: %f" % score)
        print()
        print("Q: How many killing events that involve %s happened in %s?" % (c.sf[1], c.sf[0]))
        print("A: %d" % c.num_answer_e)
        print("###########################################################################")
        print()
        

compute_confusion_metrics(look_up,CONFUSION_TUPLE,candidates)

CANDIDATE: ('Daytona Beach', 'Rakim Watson')
The location Daytona Beach has 1 meanings over 3 incidents. OA is 1, Noise-to-Signal ratio is 1.500000
The participant Rakim Watson has 0 meanings over 0 incidents. OA is 0, Noise-to-Signal ratio is 0.000000
Question score: 0.625000

Q: How many killing events that involve Rakim Watson happened in Daytona Beach?
A: 2
###########################################################################

CANDIDATE: ('Cleveland', 'Michael')
The location Cleveland has 1 meanings over 4 incidents. OA is 1, Noise-to-Signal ratio is 2.000000
The participant Michael has 12 meanings over 12 incidents. OA is 12, Noise-to-Signal ratio is 6.000000
Question score: 5.250000

Q: How many killing events that involve Michael happened in Cleveland?
A: 2
###########################################################################

CANDIDATE: ('Brooklyn', 'Ronald')
The location Brooklyn has 1 meanings over 5 incidents. OA is 1, Noise-to-Signal ratio is 2.500000
The partic